In [3]:
# Importation des bibliothèques nécessaires
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

df_offres = pd.read_excel('Data/df_clean_offres.xlsx')
df_profils = pd.read_csv('Data/profile_clean.csv')

# Chargement du modèle BERT multilingue rapide
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Préparation des champs textuels candidats
df_profils['full_profil'] = (
    "[PROFIL] " + df_profils['Profil'].fillna('') + '\n' +
    "[POINTS_FORTS] " + df_profils['Points_forts'].fillna('') + '\n' +
    "[COMPETENCES] " + df_profils['Compétence'].fillna('') + '\n' +
    "[EXPERIENCE] " + df_profils['Expérience'].fillna('')
)

# Préparation des champs textuels offres par blocs
def get_text_blocks(row):
    return {
        'poste': row['Nom_poste'],
        'contrat': row['Contrat'],
        'profil': row.get('profil', ''),
        'stack': row.get('stack_technique', ''),
        'missions': row.get('missions', ''),
        'fallback': row.get('Description', '')
    }

# Encodage des profils candidats
embeddings_profils = model.encode(df_profils['full_profil'].tolist(), convert_to_tensor=True)

# Fonction de matching BERT pour une seule offre
def match_une_offre(offre_row):
    texte_offre = get_text_blocks(offre_row)

    blocs = {
        'poste': texte_offre['poste'],
        'profil': texte_offre['profil'],
        'stack': texte_offre['stack'],
        'missions': texte_offre['missions'],
        'fallback': texte_offre['fallback']
    }

    # Si une section est vide, utiliser la description comme fallback
    for k in ['profil', 'stack', 'missions']:
        if pd.isna(blocs[k]) or not str(blocs[k]).strip():
            blocs[k] = blocs['fallback']
        else:
            blocs[k] = str(blocs[k])

    # Création du texte complet par profil d’offre
    full_offre = (
        "[POSTE] " + blocs['poste'] + '\n' +
        "[PROFIL] " + blocs['profil'] + '\n' +
        "[STACK] " + blocs['stack'] + '\n' +
        "[MISSIONS] " + blocs['missions']
    )

    emb_offre = model.encode(full_offre, convert_to_tensor=True)

    # Similarité cosinus entre l'offre et tous les profils
    scores = util.cos_sim(emb_offre, embeddings_profils)[0].cpu().numpy()

    # Top 5 profils
    top_indices = np.argsort(scores)[::-1][:5]

    top_resultats = []
    for i in top_indices:
        top_resultats.append({
            'Offre': offre_row['Nom_poste'],
            'Profil_candidat': df_profils.iloc[i]['Profil'],
            'Score_similarité': round(float(scores[i]), 3),
            'Contrat_offre': offre_row['Contrat'],
            'Contrat_candidat': df_profils.iloc[i].get('Contrat', ''),
            'Metier_candidat': df_profils.iloc[i].get('Metier_regroupe', ''),
            'Matching_contrat': int(offre_row['Contrat'] == df_profils.iloc[i].get('Contrat', ''))
        })

    return top_resultats

# Exemple : matching pour la première offre
top_matches_example = match_une_offre(df_offres.iloc[1])
top_matches_example


[{'Offre': 'Ingénieur base de données (H/F)',
  'Profil_candidat': "A la recherche d'un poste d'épidémiologiste ou de data manager",
  'Score_similarité': 0.77,
  'Contrat_offre': 'CDD',
  'Contrat_candidat': nan,
  'Metier_candidat': 'Autre',
  'Matching_contrat': 0},
 {'Offre': 'Ingénieur base de données (H/F)',
  'Profil_candidat': "A la recherche d'un poste d'épidémiologiste ou de data manager",
  'Score_similarité': 0.77,
  'Contrat_offre': 'CDD',
  'Contrat_candidat': nan,
  'Metier_candidat': 'Autre',
  'Matching_contrat': 0},
 {'Offre': 'Ingénieur base de données (H/F)',
  'Profil_candidat': 'Docteur en Data science/ IA appliqué en santé',
  'Score_similarité': 0.737,
  'Contrat_offre': 'CDD',
  'Contrat_candidat': nan,
  'Metier_candidat': 'Autre',
  'Matching_contrat': 0},
 {'Offre': 'Ingénieur base de données (H/F)',
  'Profil_candidat': 'Docteur en Data science/ IA appliqué en santé',
  'Score_similarité': 0.737,
  'Contrat_offre': 'CDD',
  'Contrat_candidat': nan,
  'Metie